# `InputWriter` API

## Getting started

In [1]:
from input_writer import InputWriter

Create an `InputWriter` instance with the name of the input file.

In [2]:
input_writer = InputWriter("dphqq.d")

The `InputWriter` has already located and saved the current coefficients.

In [3]:
for state in input_writer.states:
    for i in input_writer.line_numbers[state]:
        print(f"{i}:    {input_writer.lines[i]!r}")

65:    'c01  1s0     0.126  0.        138.039   0.\n'
67:    'c01  1s0     0.3048 0.        138.039   0.\n'
69:    'c00  1p1    13.319  0.        138.039   0.\n'
71:    'c00  1p1     0.849  0.        138.039   0.\n'
73:    'c10  3s1   -0.5837950.        138.039   0.\n'
75:    'c10  3s1   0.2399   0.        138.039   0.\n'
77:    's120 3s1   0.00     0.        138.039   0.\n'
79:    'ls0  3s1   0.0      0.        138.039   0.\n'
81:    'c11  3pj   1.28     0.        138.039   0.\n'
83:    'c11  3pj  -0.520    0.        138.039   0.\n'
85:    'ls1  3pj  -0.865    0.        138.039   0.\n'
87:    's121 3pj   0.00     0.        138.039   0.\n'


A copy of all coefficients can be accessed in a structured format via the `coefficients` property. Note that modifying this structure ___does not___ modify the input file.

In [4]:
input_writer.coefficients

{'1S0': (0.126, 0.3048),
 '1P1': (13.319, 0.849),
 '3S1': (-0.583795, 0.2399, 0.0, 0.0),
 '3PJ': (1.28, -0.52, -0.865, 0.0)}

## Preparing for an optimization run

`InputWriter` instances must be told which states are being targeted for optimization. This is done by assigning a list of states to the `target_states` property.

In [5]:
input_writer.target_states = ["1S0", "1P1"]

__Note:__ if the input file has not been configured to compute phase shifts which depend on these coefficients, the optimization ___will still be attempted___. It is up to the user to properly specify `jb,je` and `sing,tr,cp` in the input file.

We can view which coefficients are being targeted with the `target_coefficients` property. Notice that this returns a list instead of a dictionary; the coefficients are ordered according to the order of states in `target_states`.

In [6]:
input_writer.target_coefficients

[0.126, 0.3048, 13.319, 0.849]

To demonstrate injecting new coefficients into the input file, suppose that we want to randomly perturb the current target coefficients. Remember that a list of the current values of the target coefficients can be obtained with the `target_coefficients` property.

In [7]:
import numpy as np

In [8]:
perturbed_coefficients = [
    x * np.random.normal(loc=1.00, scale=0.10)
    for x in input_writer.target_coefficients
]
print(perturbed_coefficients)

[0.10826209540328971, 0.3492327579868452, 15.034979115137558, 0.9700066654702455]


Our new coefficients can then be injected using the `modify_coefficients()` method. Note that this method expects a list of floats of the same size as `target_coefficients`.

In [9]:
input_writer.modify_coefficients(perturbed_coefficients)

It's important to note that our changes have not yet been saved to the file.

Let's preview our changes, just for fun.

In [10]:
for state in input_writer.target_states:
    for i in input_writer.line_numbers[state]:
        print(f"{i}:    {input_writer.lines[i]!r}")

65:    'c01  1s0   0.108262 0.        138.039   0.\n'
67:    'c01  1s0   0.349233 0.        138.039   0.\n'
69:    'c00  1p1   15.0349790.        138.039   0.\n'
71:    'c00  1p1   0.970007 0.        138.039   0.\n'


We can see on line `69` that our coefficient collides with the next column. While this isn't ideal, both the Python scripts and Fortran executables are able to properly handle this issue.

To save these new coefficients to the file, we use the `write_lines()` method.

In [11]:
input_writer.write_lines()

Reading the file from disk, we can indeed see our new coefficients.

In [12]:
!cat dphqq.d

input for phases
jb,je       0  0
jborn       9
nj         32 
sing,tr,cp  1  0  0
iprop       1
iphrel      1
ipotfc
c         1000.       0.
wn        938.9183  938.9183
          0.01
          0.02
          0.03
          1.
          5.
          10.
          25.
          50.
          100.
          150.
          200.
          250.
          300.
elab end
mps,analy   0  0  0  0  0  1  0
icpp        0  0
icoul      80
rcoul     10.
irma,iqua   0  0
ipoint      0
iwrite      2
ipunch      0  2
ideg        2
parameters for bessel transformation
nn        240
ax,bx,cdex   0.0    5.        35.
qcut       5000.
x308b
MN Lamb    938.9183    0.
ga,fpi       1.29   92.4
c_i OUR      -0.74  0.00      -3.61     2.44
d_i         
cutall       7.               4.        1.1 
ss   opep    1.29   92.4      134.9766  0.
fun          1.
s12  opep    1.29   92.4      134.9766  0.
fun          2.
ss   opep    1.29   92.4      139.5702  0.
fun          1.
s12  opep    1.29   92.4      139.5702 

In [13]:
!git checkout -- dphqq.d